In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uuuuuu/train_folds.csv
/kaggle/input/uuuuuu/train_dataset.csv
/kaggle/input/uuuuuu/test_dataset.csv


In [2]:

pip install wtfml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import torch
import albumentations
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F
from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

In [4]:
import torch
from torch.utils.data import Dataset
import numpy as np
import cv2
import albumentations as A

class CustomImageDataset(Dataset):
    def __init__(self, pixel_arrays, targets, resize=None, augmentations=None):
        self.pixel_arrays = pixel_arrays
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        if self.face_cascade.empty():
            raise IOError('Failed to load Haar cascade file.')

    def __len__(self):
        return len(self.pixel_arrays)

    def preprocess_image(self, image):
        # Convert the 1D array to a 2D grayscale image
        image = np.array(image, dtype=np.float32).reshape(48,48)
        
        # Convert to 8-bit grayscale image
        image = (image * 255).astype(np.uint8)  # Assuming the original image is normalized
        
        # Ensure image is in grayscale
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Detect face(s)
        faces = self.face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        if len(faces) == 0:
            # If no faces detected, return original image or handle it as required
            return image
        
        # Extract the first detected face (or handle multiple faces as needed)
        (x, y, w, h) = faces[0]
        face = image[y:y+h, x:x+w]
        
        # Resize the extracted face region to the desired size
        face = cv2.resize(face, (224, 244))
        
        return face

    def __getitem__(self, idx):
        image = self.preprocess_image(self.pixel_arrays[idx])
        
        # Convert to RGB if necessary (add an extra channel dimension)
        if len(image.shape) == 2:
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)  # Convert grayscale to RGB

        # Apply augmentations if specified
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        
        # Convert image to PyTorch tensor and permute to (3, new_size, new_size) format
        image = torch.tensor(image, dtype=torch.float32)  # Create tensor
        if len(image.shape) == 3:
            image = image.permute(2, 0, 1)  # For RGB, shape becomes (3, new_size, new_size)
        elif len(image.shape) == 2:
            # Add a channel dimension (1) if grayscale
            image = image.unsqueeze(0)  # (1, new_size, new_size)
            image = image.expand(3, -1, -1)  # Expand to (3, new_size, new_size)
        
        target = torch.tensor(self.targets[idx], dtype=torch.long)
        
        return image, target
import torch
import numpy as np
from sklearn import metrics
from tqdm import tqdm

class Engine:
    @staticmethod
    def train(data_loader, model, optimizer, device, scheduler=None, accumulation_steps=1, fp16=False):
        model.train()
        losses = AverageMeter()
        scaler = torch.cuda.amp.GradScaler() if fp16 else None
        
        if accumulation_steps > 1:
            optimizer.zero_grad()
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for batch_idx, (images, targets) in enumerate(tk0):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                if fp16:
                    with torch.cuda.amp.autocast():
                        outputs = model(images)
                        loss = FocalLoss()(outputs, targets)
                    scaler.scale(loss).backward()
                else:
                    outputs = model(images)
                    loss = FocalLoss()(outputs, targets)
                    loss.backward()

                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer) if fp16 else optimizer.step()
                    if scheduler:
                        scheduler.step()
                    optimizer.zero_grad()

            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)
        
        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                images, targets = data  # Adjust if your data format is different

                # Move tensors to device
                images = images.to(device)
                targets = targets.to(device)

                # Forward pass
                predictions, loss = model(images, targets)
                predictions = predictions.cpu()
                losses.update(loss.item(), images.size(0))
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)

        # Concatenate all predictions and convert to NumPy array
        final_predictions = torch.cat(final_predictions).numpy()
        return final_predictions, losses.avg
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                inputs, _ = data  # Unpack data
                inputs = inputs.to(device)
                predictions = model(inputs) 
                predictions = F.softmax(predictions,dim = 1)# Assume model returns only predictions
                final_predictions.append(predictions.cpu())
                tk0.set_postfix()
        return torch.cat(final_predictions).numpy()
        # Concatenate all predictions and convert to numpy array
        return torch.cat(final_predictions).numpy()
class AverageMeter():
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count

In [5]:
import albumentations as A
from torch.utils.data import DataLoader
def train(fold):
    df = pd.read_csv("/kaggle/input/uuuuuu/train_folds.csv")
    device = "cuda"
    epochs = 50
    train_bs = 16
    valid_bs = 16

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    train_pixels = df_train['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    train_targets = df_train['emotion'].values

    valid_pixels = df_valid['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    valid_targets = df_valid['emotion'].values

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = A.Compose(
    [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
        A.HorizontalFlip(p=0.5)
    ]
    )

    valid_aug = A.Compose(
        [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True)
        ]
    )
    train_dataset = CustomImageDataset(
        pixel_arrays=train_pixels,
        targets=train_targets,
        resize=(299, 299),  # New size
        augmentations=train_aug,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=16, shuffle=True, num_workers=4
    )

    valid_dataset = CustomImageDataset(
        pixel_arrays=valid_pixels,
        targets=valid_targets,
        resize=(299, 299),  # New size
        augmentations=valid_aug,
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=16, shuffle=False, num_workers=4
    )
    model = EF(num_classes= 7,pretrained='imagenet')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")
    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model,optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        fp16=False)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        final_predictions = np.argmax(np.vstack(predictions), axis=1)
        valid_targets = np.array(valid_targets)  # Ensure targets are in the right format
    
    # Calculate accuracy
        accuracy = metrics.accuracy_score(valid_targets, final_predictions)
        print(f"Epoch = {epoch}, Accuracy = {accuracy:.4f}")
    
    # Update the learning rate scheduler
        scheduler.step(accuracy)
    
    # Early Stopping
        es(accuracy, model, model_path=f"model_fold_EFb6F{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break
    oof_data = {
        'id': df_valid.index,  # Use the index to map back to the original data
        'true_emotion': valid_targets,
        'pred_emotion': final_predictions,
        'fold': fold
    }
    
    return pd.DataFrame(oof_data)


In [6]:
pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=faaa0fa83a81a7825b13f51dd0b84c758ffb34d6d7023a5d2598cd4964ac17dc
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from torch import nn
from efficientnet_pytorch import EfficientNet

class EF(nn.Module):
    def __init__(self, num_classes=7, model_name='efficientnet-b6', pretrained='imagenet'):
        super(EF, self).__init__()
        
        # Load the pre-trained EfficientNet model
        self.base_model = EfficientNet.from_pretrained(model_name) if pretrained == 'imagenet' else EfficientNet.from_name(model_name)
        
        # Replace the final fully connected layer with the new layer for multi-class classification
        in_features = self.base_model._fc.in_features
        self.base_model._fc = nn.Linear(in_features, num_classes)
    
    def forward(self, image, targets=None):
        batch_size = image.size(0)
        
        # Forward pass through the base model
        out = self.base_model(image)
        
        # If targets are provided, calculate the loss
        if targets is not None:
            loss = FocalLoss()(out, targets)
            return out, loss
        
        return out

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from NaN-loss errors

    def forward(self, logits, target):
        """
        logits: [batch_size, num_classes]
        target: [batch_size] (true class indices, not one-hot)
        """
        # Compute softmax over logits to get the class probabilities
        probs = F.softmax(logits, dim=1)
        
        # Gather the probabilities of the true class (index-based selection)
        probs_target_class = probs.gather(1, target.unsqueeze(1)).squeeze(1) + self.epsilon
        
        # Compute the log of the selected probabilities
        log_pt = torch.log(probs_target_class)
        
        # Calculate the focal loss (focuses more on hard examples)
        focal_loss = -1 * self.alpha * (1 - probs_target_class) ** self.gamma * log_pt
        
        return torch.mean(focal_loss)

In [9]:
X = np.empty((0, 4))  # Initialize an empty array with 7 columns to match df.values

for i in range(0, 5):
    df = train(i)  # Assuming train(1) returns a DataFrame with 7 columns
    df.to_csv(f"oof1EFOCAL-{i}.csv", index=False, header=False)
    if len(df.columns)!=4:
        df = df.drop('Unnamed: 0',axis = 1)    
    X = np.concatenate((X, df.values))  # Concatenate new data to X

# Convert the final result into a DataFrame and save to CSV
df = pd.DataFrame(X)
df.to_csv('EFFb6.csv', index=False)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth
100%|██████████| 165M/165M [00:00<00:00, 260MB/s]


Loaded pretrained weights for efficientnet-b6


100%|██████████| 63/63 [00:16<00:00,  3.79it/s, loss=0.891]


Epoch = 0, Accuracy = 0.5010
Validation score improved (-inf --> 0.501). Saving model!


100%|██████████| 63/63 [00:16<00:00,  3.77it/s, loss=0.726]


...

Epoch = 24, Accuracy = 0.5920
EarlyStopping counter: 3 out of 5


100%|██████████| 63/63 [00:16<00:00,  3.76it/s, loss=1.2]


Epoch = 25, Accuracy = 0.5950
EarlyStopping counter: 4 out of 5


100%|██████████| 63/63 [00:16<00:00,  3.72it/s, loss=1.2]

Epoch = 26, Accuracy = 0.5950
EarlyStopping counter: 5 out of 5
Early stopping


In [10]:
def predict(fold):
    df = pd.read_csv("/kaggle/input/uuuuuu/test_dataset.csv")
    device = "cuda"
    model_path=f"model_fold_EFb6F{fold}.bin"
    test_pixels = df['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))                 
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    aug = albumentations.Compose(
        [
            albumentations.Resize(height=299, width=299),
            albumentations.Normalize(mean, std, max_pixel_value=255.0, always_apply=True)
        ]
    )
                                      
    targets = np.zeros(len(df))
    test_dataset = CustomImageDataset(
        pixel_arrays=test_pixels,
        targets=targets,
        resize=None,
        augmentations=aug,
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4
    )

    model = model = EF(pretrained='imagenet')
    model.load_state_dict(torch.load(model_path))
    model.to(device)

    predictions = Engine.predict(test_loader, model, device=device)
    predictions = np.vstack((predictions))
    return predictions


In [11]:
P1 = predict(0)
P2 = predict(1)
P3 = predict(2)
P4 = predict(3)
P5 = predict(4)
P = (P1 + P2 +P3 +P4 + P5)/5
df = pd.DataFrame(P)
df.to_csv('EFFb6.csv')


Loaded pretrained weights for efficientnet-b6


/tmp/ipykernel_23/4232482971.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|██████████| 157/157 [00:41<00:00,  3.81i

Loaded pretrained weights for efficientnet-b6


/tmp/ipykernel_23/4232482971.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|██████████| 157/157 [00:40<00:00,  3.84i

Loaded pretrained weights for efficientnet-b6


/tmp/ipykernel_23/4232482971.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|██████████| 157/157 [00:41<00:00,  3.83i

Loaded pretrained weights for efficientnet-b6


/tmp/ipykernel_23/4232482971.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|██████████| 157/157 [00:40<00:00,  3.84i

Loaded pretrained weights for efficientnet-b6


/tmp/ipykernel_23/4232482971.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))
100%|██████████| 157/157 [00:41<00:00,  3.82i